# Tutorial 07 - Nonlinear elliptic problems

In [ ]:
import typing

In [ ]:
import dolfinx.fem
import dolfinx.fem.petsc
import dolfinx.io
import gmsh
import mpi4py.MPI
import numpy as np
import numpy.typing
import petsc4py.PETSc
import ufl
import viskex

In [ ]:
import rbnicsx.backends

## 1. Mesh generation

In [ ]:
mesh_size = 5e-2

In [ ]:
gmsh.initialize()
gmsh.model.add("nonlinear_elliptic")

In [ ]:
p0 = gmsh.model.geo.addPoint(0.0, 0.0, 0.0, mesh_size)
p1 = gmsh.model.geo.addPoint(1.0, 0.0, 0.0, mesh_size)
p2 = gmsh.model.geo.addPoint(1.0, 1.0, 0.0, mesh_size)
p3 = gmsh.model.geo.addPoint(0.0, 1.0, 0.0, mesh_size)
l0 = gmsh.model.geo.addLine(p0, p1)
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p0)
boundary = gmsh.model.geo.addCurveLoop([l0, l1, l2, l3])
domain = gmsh.model.geo.addPlaneSurface([boundary])

In [ ]:
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, [l0, l1, l2, l3], 1)
gmsh.model.addPhysicalGroup(2, [domain], 1)
gmsh.model.mesh.generate(2)

In [ ]:
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)
gmsh.finalize()

In [ ]:
viskex.dolfinx.plot_mesh(mesh)

In [ ]:
viskex.dolfinx.plot_mesh_tags(mesh, subdomains, "subdomains")

In [ ]:
viskex.dolfinx.plot_mesh_tags(mesh, boundaries, "boundaries")

## 2. Problem definition

In [ ]:
class Problem(object):
    """Define a nonlinear problem, and solve it with SNES."""

    def __init__(self) -> None:
        # Define function space
        V = dolfinx.fem.functionspace(mesh, ("Lagrange", 1))
        self._V = V
        # Define test function
        v = ufl.TestFunction(V)
        # Define solution
        u = dolfinx.fem.Function(V)
        self._solution = u
        # Define symbolic parameters for use in UFL forms
        mu_symb = rbnicsx.backends.SymbolicParameters(mesh, shape=(2, ))
        self._mu_symb = mu_symb
        # Define residual form of the problem
        x = ufl.SpatialCoordinate(mesh)
        g = 100 * ufl.sin(2 * ufl.pi * x[0]) * ufl.sin(2 * ufl.pi * x[1])
        residual = (
            ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
            + ufl.inner(mu_symb[0] * (ufl.exp(mu_symb[1] * u) - 1) / mu_symb[1], v) * ufl.dx
            - ufl.inner(g, v) * ufl.dx
        )
        self._residual = residual
        self._residual_cpp = dolfinx.fem.form(residual)
        # Define jacobian form of the problem
        jacobian = ufl.derivative(residual, u)
        self._jacobian = jacobian
        self._jacobian_cpp = dolfinx.fem.form(jacobian)
        # Define boundary conditions for the problem
        zero = petsc4py.PETSc.ScalarType(0)
        bdofs_V = dolfinx.fem.locate_dofs_topological(V, mesh.topology.dim - 1, boundaries.indices)
        bcs = [dolfinx.fem.dirichletbc(zero, bdofs_V, V)]
        self._bcs = bcs

    @property
    def function_space(self) -> dolfinx.fem.FunctionSpaceBase:
        """Return the function space of the problem."""
        return self._V

    @property
    def solution(self) -> dolfinx.fem.Function:
        """Return the solution of the problem for the latest parameter value."""
        return self._solution

    @property
    def residual_form(self) -> ufl.Form:  # type: ignore[no-any-unimported]
        """Return the residual form of the problem."""
        return self._residual

    @property
    def jacobian_form(self) -> ufl.Form:  # type: ignore[no-any-unimported]
        """Return the jacobian form of the problem."""
        return self._jacobian

    @property
    def boundary_conditions(self) -> typing.List[dolfinx.fem.DirichletBC]:
        """Return the boundary conditions for the problem."""
        return self._bcs

    def update_solution(self, x: petsc4py.PETSc.Vec) -> None:  # type: ignore[no-any-unimported]
        """Update `self._solution` with data in `x`."""
        x.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
        with x.localForm() as _x, self._solution.vector.localForm() as _solution:
            _solution[:] = _x

    def _assemble_residual(  # type: ignore[no-any-unimported]
        self, snes: petsc4py.PETSc.SNES, x: petsc4py.PETSc.Vec, residual_vec: petsc4py.PETSc.Vec
    ) -> None:
        """Assemble the residual."""
        self.update_solution(x)
        with residual_vec.localForm() as residual_vec_local:
            residual_vec_local.set(0.0)
        dolfinx.fem.petsc.assemble_vector(residual_vec, self._residual_cpp)
        residual_vec.ghostUpdate(addv=petsc4py.PETSc.InsertMode.ADD, mode=petsc4py.PETSc.ScatterMode.REVERSE)
        dolfinx.fem.petsc.set_bc(residual_vec, self._bcs, x, -1.0)

    def _assemble_jacobian(  # type: ignore[no-any-unimported]
        self, snes: petsc4py.PETSc.SNES, x: petsc4py.PETSc.Vec, jacobian_mat: petsc4py.PETSc.Mat,
        preconditioner_mat: petsc4py.PETSc.Mat
    ) -> None:
        """Assemble the jacobian."""
        jacobian_mat.zeroEntries()
        dolfinx.fem.petsc.assemble_matrix(  # type: ignore[misc]
            jacobian_mat, self._jacobian_cpp, self._bcs, diagonal=1.0)  # type: ignore[arg-type]
        jacobian_mat.assemble()

    def solve(self, mu: np.typing.NDArray[np.float64]) -> dolfinx.fem.Function:
        """Assign the provided parameters value and solve the problem."""
        self._mu_symb.value[:] = mu
        return self._solve()

    def _solve(self) -> dolfinx.fem.Function:
        """Solve the nonlinear problem with SNES."""
        snes = petsc4py.PETSc.SNES().create(mesh.comm)
        snes.setTolerances(max_it=20)
        snes.getKSP().setType("preonly")
        snes.getKSP().getPC().setType("lu")
        snes.getKSP().getPC().setFactorSolverType("mumps")
        residual_vec = dolfinx.fem.petsc.create_vector(self._residual_cpp)
        snes.setFunction(self._assemble_residual, residual_vec)
        jacobian_mat = dolfinx.fem.petsc.create_matrix(self._jacobian_cpp)
        snes.setJacobian(self._assemble_jacobian, J=jacobian_mat, P=None)
        snes.setMonitor(lambda _, it, residual: print(it, residual))
        solution_copy = self._solution.vector.copy()
        snes.solve(None, solution_copy)
        self.update_solution(solution_copy)
        residual_vec.destroy()
        jacobian_mat.destroy()
        solution_copy.destroy()
        snes.destroy()
        return self._solution

In [ ]:
problem = Problem()

In [ ]:
mu_solve = np.array([0.3, 9.0])
solution = problem.solve(mu_solve)

In [ ]:
viskex.dolfinx.plot_scalar_field(solution, "high fidelity solution")